In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The final return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
          'version': '2',  
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The final return value for a "way" element should look something like:

{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}
"""

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

# for project
OSM_PATH = "project_map.osm"

# for test audit.py
# OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

EXPECTED = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Way", "Broadway", "Colfax"]

mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Ave." : "Avenue",
            "Rd" : "Road", 
            "Rd." : "Road",
            "Blvd" : "Boulevard",
            "Blvd." : "Boulevard",
            "Cir" : "Circle",
            "Cir." : "Circle",
            "Ct" : "Court",
            "Ct." : "Court",
            "Dr" : "Drive",
            "Dr." : "Drive",
            "Ln" : "Lane",
            "Ln." : "Lane",
            "Pl" : "Place",
            "Pl." : "Place",
            "Pky": "Parkway",
            "Pky.": "Parkway",
            "Ter": "Terrace",
            "Ter.": "Terrace",
            "W" : "West",
            "N" : "North",
            "S" : "South",
            "E" : "East",
            "W." : "West",
            "N." : "North",
            "S." : "South",
            "E." : "East"
            }

def source_update(value):
    if value in ["Bing", "bing_imagery"]:
        return "bing_imagery"
    elif value == "usgs_imagery;survey;image":
        return "survey;image;usgs_imagery"
    else:
        return value

def tiger_update(key, value, mapping, EXPECTED):
    if key == "name_direction" or key == "name_type":
        if value in mapping.keys():
            return mapping[value]
        elif value in EXPECTED:
            return value
        else: # debug for not detected  mapping value
            print key, value 
            return value
    else:
        return value

    
def addr_update(street, mapping, EXPECTED):
    name_direction = None
    name_type = None
    
    # change to regular expression to pick out the number part
    street_type_re = re.compile(r'\d*\s*([a-zA-Z0-9_ ]+\.?$)')
    m = street_type_re.search(street)
    if m:
        street_name = m.group()
    else:
        return name_direction, name_typ 
    
    # retrieving street type
    street_list = street_name.split(" ")
    if street_list[-1] in EXPECTED:
        name_type = street_list[-1]
    elif street_list[-1] in mapping.keys():
        name_type = mapping[street_list[-1]]
    
    street_out_type = (" ").join(street_list[:-1])
    if street_list[0] in ['South', 'North', 'West', 'East']:
        name_direction = street_list[0]
    elif street_list[0] in mapping.keys():
        name_direction = mapping[street_list[0]]
    
    return name_direction, name_type

def fill_dict(dict_id, dict_type, dict_key, dict_value):
    newdict = {}
    newdict['id'] = dict_id
    newdict['type'] = dict_type
    newdict['key'] = dict_key 
    newdict['value'] = dict_value
    return newdict
    
def load_secondary(element, secondary, default_tag_type):
    newdict_list = []
    newdict = {}
    newdict['id'] = element.attrib['id']
    
    if ':' in secondary.attrib['k']:
        #print secondary.attrib['k'], secondary.attrib['v']
        address_list = secondary.attrib['k'].split(':')
        if len(address_list) == 2:
            newdict['type'] = address_list[0]
            if address_list[1] == "name_direction_prefix":
                newdict['key'] = "name_direction"
            else:
                newdict['key'] = address_list[1]
        elif len(address_list) > 2:
            newdict['type'] = address_list[0]
            newdict['key'] = (":").join(address_list[1:])
        else:
            pass
    else:
        newdict['type'] = default_tag_type
        newdict['key'] = secondary.attrib['k']
    
    if newdict['type'] =="tiger":
        newdict['value'] = tiger_update(newdict['key'], secondary.attrib['v'], mapping, EXPECTED)
        #print newdict['type'], newdict['key'],newdict['value']
    elif newdict['key'] == "source":
        newdict['value'] = source_update(secondary.attrib['v'])
    else:
        newdict['value'] = secondary.attrib['v']
    newdict_list.append(newdict)
    
    # split the type address, key = street part.
    if newdict['type'] == "addr" and newdict['key'] =="street":
        name_direction, name_type = addr_update(newdict['value'], mapping, EXPECTED)
        if name_type:
            addrdict = fill_dict(element.attrib['id'], "addr", "name_type", name_type)
            newdict_list.append(addrdict)
            #print addrdict['type'], addrdict['key'],addrdict['value']
            
        if name_direction:
            addrdict = fill_dict(element.attrib['id'], "addr", "name_direction", name_direction)
            newdict_list.append(addrdict)
            #print addrdict['type'], addrdict['key'],addrdict['value']
            #addrdict.clear()
        
    return newdict_list

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for attrib, value in element.attrib.iteritems():
            if attrib in node_attr_fields:
                node_attribs[attrib] = value
        for secondary in element.iter():
            if secondary.tag == 'tag':
                if problem_chars.match(secondary.attrib['k']):
                    continue
                else:
                    second_dict_list  = list()
                    second_dict_list = load_secondary(element, secondary,default_tag_type)
                    for second_dict in second_dict_list:
                        tags.append(second_dict)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for attrib, value in element.attrib.iteritems():
            if attrib in way_attr_fields:
                way_attribs[attrib] = value
        counter = 0
        for secondary in element.iter():
            if secondary.tag == 'tag':
                if problem_chars.match(secondary.attrib['k']):
                    continue
                else:
                    second_dict_list  = list()
                    second_dict_list = load_secondary(element, secondary,default_tag_type)
                    for second_dict in second_dict_list:
                        tags.append(second_dict)
            elif secondary.tag == 'nd':
                newnd = {}
                newnd['id'] = element.attrib['id']
                newnd['node_id'] = secondary.attrib['ref']
                newnd['position'] = counter
                counter += 1
                way_nodes.append(newnd)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)
